In [3]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import scripts.prediction_utils as predict_u
import scripts.theodolite_utils as theodo_u
import scripts.theodolite_plot_function as theodo_p
import scripts.groundtruth_utils as theodo_g
import scripts.theodolite_function as theodo_f
import time
import torch
from numpy import linalg
import importlib
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
theodo_f = importlib.reload(theodo_f)
predict_u = importlib.reload(predict_u)

In [5]:
%matplotlib notebook
theodo_u = importlib.reload(theodo_u)
theodo_p = importlib.reload(theodo_p)
predict_u = importlib.reload(predict_u)
theodo_g = importlib.reload(theodo_g)

## Path of main files
path = "../data/20220224/"

# Reading sensor extrinsic calibration
file_sensors = theodo_u.if_file_exist(path + "sensors_extrinsic_calibration/calibration_results.csv",'')
extrinsic_calibration_results = theodo_u.read_extrinsic_calibration_results_file(file_sensors)

# Reading sensor data
# Sensor = "Robosense_32"
Sensor = "Fake"   # GNSS or Robosense_32 or Fake
# path_sensor_file = path+"ICP/icp_odom.csv"
rate_fake = 20  # Hz
Sensor_fake = "GNSS"

# Sensor = "GNSS"
path_sensor_file = path+"gps_data/"+"gps3.txt"
path_sensor_file_synch_time = path+"gps_data/delay_synchronization_GNSS_3.txt"  # If applied to GNSS
Gps_reference_chosen = 3    # 1: front, 2: back, 3: middle   # Only for GNSS

## Parameters for pre-processing pipeline
file = [
        "/home/maxime/data/ICRA_2023/Vaidis2022_dataset/20220224/20220224_inter_prism.bag"
]
parameters = [
    [1,2,1,1,3,2],  # 1. Apply filtering or not (Module 1), 2-3-4. Parameters tau_r, tau_a, tau_e (Module 1), 5. Parameter tau_s (Module 2), 6 Parameter tau_l (Module 4).
]
output = [
        path
]

## Read sensor data which we want a ground truth
sensor_data = []
if Sensor == "GNSS":
    GNSS_raw_data = theodo_u.read_prediction_data_Linear_csv_file(path_sensor_file)
    time_delay = float(theodo_u.read_time_delay(path_sensor_file_synch_time))

    for i in GNSS_raw_data:
        raw_data = np.array([i[0] + time_delay, i[1], i[2], i[3], 0, 0, 0, 1])
        sensor_data.append(raw_data)
    sensor_data = np.array(sensor_data)

if Sensor == "Robosense_32":
    sensor_data = theodo_u.read_icp_odom_file(path_sensor_file)
    sensor_data = np.array(sensor_data)

# Limit acceptable for ground truth error
limit_dist = 100  # Need to check the impact
limit_uncertainty = 100 # Limit of eigenvalues taken into account

## Range
random_noise_range = [0, 0.004/2, 2]   ## Mean, sigma, ppm,  4mm + 2ppm (2 sigma)  ISO17123-3
## Angles
random_noise_angle = [0, 0.000024241/5*4/2]    # Mean, sigma, 5"=0.000024241 precision datasheet  (2 sigma)  ISO17123-3
## Tilt compensator
random_noise_tilt = [0, 0.000002424/2]    # Mean, sigma, 0.5"=0.000002424 precision datasheet  (2 sigma)  ISO17123-3
## Weather
weather_data_path = "../data/weather_2022/Quebec/"
data_weather = np.array(theodo_u.read_weather_data(weather_data_path+"data_sorted_2022.txt"))

## Number of rigid transforms to apply the uncertainty
num_samples = 1000
path_file_GCP = "total_stations/GCP.txt"
frame_chosen, T_1_corrected, T_2_corrected, T_3_corrected = theodo_g.extrinsic_calibration_noise(path+path_file_GCP, random_noise_range, random_noise_angle, random_noise_tilt, num_samples)
## Read Extrinsic results for RTS
## Estimated extrinsic calibration uncertainty
_, _, _, Tf_1, Tf_2, Tf_3 = theodo_u.read_marker_file(path+path_file_GCP, 1, 1)

## Choice of noise model
model_chosen = [1, 1, 1, 0, 0]  # Tilt noise, Atmospheric correction, extrinsic calibration, Weather corrections, Time synchronization # 1: Activate, 0:Disable

save_MC_inteprolated = True
num_samples_MC_sensor = 100
want_sensor_pose = False
save_MC_sensor = False

for param in parameters:
    print(param)

    if param[0] == 0:
        filtering = False
    if param[0] == 1:
        filtering = True
    thresold_d = param[1]  ## tau_r [m/s]
    thresold_a = param[2]  ## tau_a [deg/s]
    thresold_e = param[3]  ## tau_e [deg/s]
    limit_time_interval = param[4]  ## tau_s [s]
    size_interval = param[5]    ## tau_l

    Mode = "STEAM"  ## Interpolation choice: 1. L -> Linear interpolation, 2. SGP -> Gaussian Process with Stheno library, 3. STEAM
    limit_search = limit_time_interval
    save = False

    save_index_1 = []
    save_index_2 = []
    save_index_3 = []

    for fname, opath in zip(file, output):
        if not filtering:
            path_out = opath + "raw_prediction/"
        else:
            path_out = opath + "filtered_prediction/"

        if filtering:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data_pre_filtered(
                fname)
            index_1_f = theodo_u.thresold_raw_data(t1, d1, a1, e1, thresold_d, thresold_a * 3.1415926 / 180,
                                                   thresold_e * 3.1415926 / 180, limit_time_interval)
            index_2_f = theodo_u.thresold_raw_data(t2, d2, a2, e2, thresold_d, thresold_a * 3.1415926 / 180,
                                                   thresold_e * 3.1415926 / 180, limit_time_interval)
            index_3_f = theodo_u.thresold_raw_data(t3, d3, a3, e3, thresold_d, thresold_a * 3.1415926 / 180,
                                                   thresold_e * 3.1415926 / 180, limit_time_interval)
            t1 = t1[index_1_f]
            t2 = t2[index_2_f]
            t3 = t3[index_3_f]
            tp1 = tp1[index_1_f].T
            tp2 = tp2[index_2_f].T
            tp3 = tp3[index_3_f].T
            print(len(t1), len(t2), len(t3))
        else:
            t1, t2, t3, tp1, tp2, tp3, d1, d2, d3, a1, a2, a3, e1, e2, e3 = theodo_u.read_rosbag_theodolite_without_tf_raw_data(
                fname)
            print(len(t1), len(t2), len(t3))

        list_interval, list_time = theodo_f.split_time_interval_all_data(t1, t2, t3, limit_time_interval)
        list_trajectories_split = theodo_f.merge_interval(list_interval, list_time, t1, t2, t3, limit_search)

        M_1_before_inter = []
        M_2_before_inter = []
        M_3_before_inter = []
        Prediction_1 = []
        Prediction_2 = []
        Prediction_3 = []
        T_prediction = []
        Index_sensor = []

        print("Number of sub-trajectories :", len(list_trajectories_split))
        number_ite = -1
        for i in list_trajectories_split:
            number_ite = number_ite + 1
            print("Étape " + str(number_ite))
            index_1 = np.array([i[0, 0], i[1, 0]])
            index_2 = np.array([i[0, 1], i[1, 1]])
            index_3 = np.array([i[0, 2], i[1, 2]])

            save_index_1.append(index_1)
            save_index_2.append(index_2)
            save_index_3.append(index_3)

            begin = np.max([t1[index_1[0]], t2[index_2[0]], t3[index_3[0]]])
            end = np.min([t1[index_1[1]], t2[index_2[1]], t3[index_3[1]]])

            if abs(end - begin) > size_interval and begin < end and number_ite<100:  # control number of sample

                T_prediction_sensor = []
                if Sensor!="Fake":
                    Number = 0
                    for value_sensor_data in sensor_data:
                        if end >= value_sensor_data[0] >= begin:
                            T_prediction_sensor.append(value_sensor_data[0])
                            Index_sensor.append(Number)
                        Number = Number + 1

                    T_prediction_init = torch.from_numpy(np.array(T_prediction_sensor))
                else:
                    List_time = np.arange(begin, end, 1/rate_fake)
                    for i in List_time:
                        T_prediction_sensor.append(i)
                    T_prediction_init = torch.from_numpy(np.arange(begin, end, 1/rate_fake))

                # Linear interpolation
                if Mode == "L" or Mode == "All":

                    ## Put trajectories in same frame
                    tp1 = Tf_1 @ tp1
                    tp2 = Tf_2 @ tp2
                    tp3 = Tf_3 @ tp3

                    T1, X1, Y1, Z1, T2, X2, Y2, Z2, T3, X3, Y3, Z3 = predict_u.data_training_L(t1, t2, t3,
                                                                                      tp1, tp2, tp3,
                                                                                      index_1,
                                                                                      index_2,
                                                                                      index_3)
                    mx1, my1, mz1, mx2, my2, mz2, mx3, my3, mz3 = predict_u.linear_interpolation(T1, X1, Y1,
                                                                                        Z1, T2, X2,
                                                                                        Y2, Z2, T3,
                                                                                        X3, Y3, Z3)

                    for i in T_prediction_init.numpy():
                        T_prediction.append(i)
                        P1_L, P2_L, P3_L = predict_u.linear_prediction(i, 0, mx1, my1, mz1, mx2, my2, mz2,
                                                                          mx3, my3, mz3)
                        Prediction_1.append(P1_L)
                        Prediction_2.append(P2_L)
                        Prediction_3.append(P3_L)

                if Mode == "STEAM" or Mode == "All":

                    MC_1 = []
                    MC_2 = []
                    MC_3 = []
                    tic = time.perf_counter()
                    ## Compute uncertainty
                    T1, D1, A1, E1 = predict_u.data_training_L_Raw_data(t1, d1, a1, e1, index_1)
                    T2, D2, A2, E2 = predict_u.data_training_L_Raw_data(t2, d2, a2, e2, index_2)
                    T3, D3, A3, E3 = predict_u.data_training_L_Raw_data(t3, d3, a3, e3, index_3)
                    print("Number of input: ", len(T1))

                    for i1,j1,k1,l1 in zip(T1,D1,A1,E1):
                        mu_raw_data, _,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, j1, random_noise_range, k1, l1, random_noise_angle, random_noise_tilt, Tf_1, T_1_corrected, data_weather, i1, model_chosen)
                        MC_1.append([i1,mu_raw_data,cov_matrix_simulated])
                        M_1_before_inter.append([i1,mu_raw_data,cov_matrix_simulated])
                    for i2,j2,k2,l2 in zip(T2,D2,A2,E2):
                        mu_raw_data, _,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, j2, random_noise_range, k2, l2, random_noise_angle, random_noise_tilt, Tf_2, T_2_corrected, data_weather, i2, model_chosen)
                        MC_2.append([i2,mu_raw_data,cov_matrix_simulated])
                        M_2_before_inter.append([i2,mu_raw_data,cov_matrix_simulated])
                    for i3,j3,k3,l3 in zip(T3,D3,A3,E3):
                        mu_raw_data, _,cov_matrix_simulated = theodo_g.MC_raw_data(num_samples, j3, random_noise_range, k3, l3, random_noise_angle, random_noise_tilt, Tf_3, T_3_corrected, data_weather, i3, model_chosen)
                        MC_3.append([i3,mu_raw_data,cov_matrix_simulated])
                        M_3_before_inter.append([i3,mu_raw_data,cov_matrix_simulated])

                    ## STEAM
                    MC_1_interpolated = theodo_g.STEAM_interpolation_with_covariance(T1, T_prediction_sensor, MC_1)
                    MC_2_interpolated = theodo_g.STEAM_interpolation_with_covariance(T2, T_prediction_sensor, MC_2)
                    MC_3_interpolated = theodo_g.STEAM_interpolation_with_covariance(T3, T_prediction_sensor, MC_3)
                    toc = time.perf_counter()
                    print("Time : ", round(toc-tic,2))
                    if(len(MC_1_interpolated)>0 and len(MC_2_interpolated)>0 and len(MC_3_interpolated)>0):
                        Prediction_1.append(MC_1_interpolated)
                        Prediction_2.append(MC_2_interpolated)
                        Prediction_3.append(MC_3_interpolated)
                        T_prediction.append(T_prediction_sensor)

        MC_1_inter = []
        MC_2_inter = []
        MC_3_inter = []
        for i1,j1,k1 in zip(Prediction_1,Prediction_2,Prediction_3):
            for i2,j2,k2 in zip(i1,j1,k1):
                # Check on uncertainty. If too high, remove triplet
                eig1 = np.linalg.eigvals(i2[2])**0.5
                eig2 = np.linalg.eigvals(j2[2])**0.5
                eig3 = np.linalg.eigvals(k2[2])**0.5
                if eig1[0]<limit_uncertainty and eig1[1]<limit_uncertainty and eig1[2]<limit_uncertainty \
                        and eig2[0]<limit_uncertainty and eig2[1]<limit_uncertainty and eig2[2]<limit_uncertainty \
                        and eig3[0]<limit_uncertainty and eig3[1]<limit_uncertainty and eig3[2]<limit_uncertainty:
                    MC_1_inter.append(i2)
                    MC_2_inter.append(j2)
                    MC_3_inter.append(k2)

        if save_MC_inteprolated:
            theodo_u.save_MC_interpolated_sorted(MC_1_inter, path + "uncertainty/interpolation/MC_"+str(Sensor)+"_"+str(model_chosen[0])+"_"+str(model_chosen[1])+"_"+str(model_chosen[2])
                                                 +"_"+str(model_chosen[3])+"_"+str(model_chosen[4])+"_1.csv")
            theodo_u.save_MC_interpolated_sorted(MC_2_inter, path + "uncertainty/interpolation/MC_"+str(Sensor)+"_"+str(model_chosen[0])+"_"+str(model_chosen[1])+"_"+str(model_chosen[2])
                                                 +"_"+str(model_chosen[3])+"_"+str(model_chosen[4])+"_2.csv")
            theodo_u.save_MC_interpolated_sorted(MC_3_inter, path + "uncertainty/interpolation/MC_"+str(Sensor)+"_"+str(model_chosen[0])+"_"+str(model_chosen[1])+"_"+str(model_chosen[2])
                                                 +"_"+str(model_chosen[3])+"_"+str(model_chosen[4])+"_3.csv")

        MC_1_sorted = []
        MC_2_sorted = []
        MC_3_sorted = []
        for i1,j1,k1 in zip(MC_1_inter,MC_2_inter,MC_3_inter):
            d1 = abs(np.linalg.norm(i1[1]-j1[1])-extrinsic_calibration_results[0])
            d2 = abs(np.linalg.norm(i1[1]-k1[1])-extrinsic_calibration_results[1])
            d3 = abs(np.linalg.norm(k1[1]-j1[1])-extrinsic_calibration_results[2])
            if d1<limit_dist and d2<limit_dist and d3<limit_dist:
                MC_1_sorted.append(i1)
                MC_2_sorted.append(j1)
                MC_3_sorted.append(k1)

        if want_sensor_pose==True:
            if Sensor!="Fake":
                P_sensor = theodo_g.chose_sensor_before_ptp(path, Sensor, Gps_reference_chosen)
                Pose_sensor_MC = []
                for i_mc,j_mc,k_mc in zip(MC_1_sorted,MC_2_sorted,MC_3_sorted):
                    Pose_sensor = []
                    p1_corrected = theodo_g.return_point_from_covariance(i_mc[2],i_mc[1], num_samples_MC_sensor)
                    p2_corrected = theodo_g.return_point_from_covariance(j_mc[2],j_mc[1], num_samples_MC_sensor)
                    p3_corrected = theodo_g.return_point_from_covariance(k_mc[2],k_mc[1], num_samples_MC_sensor)
                    for i,j,k in zip(p1_corrected,p2_corrected,p3_corrected):
                        Q = np.array([i, j, k]).T
                        Q =np.concatenate((Q, np.array([[1,1,1]])), axis=0)
                        T = theodo_u.point_to_point_minimization(P_sensor, Q)
                        Pose_sensor.append(T)
                    p_T, mu_T, cov_T = theodo_g.find_noise_list_tf(Pose_sensor)
                    Pose_sensor_MC.append([i_mc[0],mu_T, cov_T])
            else:
                P_sensor = theodo_g.chose_sensor_before_ptp(path, Sensor_fake, Gps_reference_chosen)
                Pose_sensor_MC = []
                for i_mc,j_mc,k_mc in zip(MC_1_sorted,MC_2_sorted,MC_3_sorted):
                    Pose_sensor = []
                    p1_corrected = theodo_g.return_point_from_covariance(i_mc[2],i_mc[1], num_samples_MC_sensor)
                    p2_corrected = theodo_g.return_point_from_covariance(j_mc[2],j_mc[1], num_samples_MC_sensor)
                    p3_corrected = theodo_g.return_point_from_covariance(k_mc[2],k_mc[1], num_samples_MC_sensor)
                    for i,j,k in zip(p1_corrected,p2_corrected,p3_corrected):
                        Q = np.array([i, j, k]).T
                        Q =np.concatenate((Q, np.array([[1,1,1]])), axis=0)
                        T = theodo_u.point_to_point_minimization(P_sensor, Q)
                        Pose_sensor.append(T)
                    p_T, mu_T, cov_T = theodo_g.find_noise_list_tf(Pose_sensor)
                    Pose_sensor_MC.append([i_mc[0],mu_T, cov_T])

            if save_MC_sensor:
                if Sensor=="GNSS":
                    theodo_u.save_MC_interpolated_sorted(Pose_sensor_MC, path + "uncertainty/sensor/"+Sensor+"_"+str(Gps_reference_chosen)+"_"
                                                         +str(model_chosen[0])+"_"+str(model_chosen[1])+"_"+str(model_chosen[2])
                                                     +"_"+str(model_chosen[3])+"_"+str(model_chosen[4])+".csv")
                else:
                    theodo_u.save_MC_interpolated_sorted(Pose_sensor_MC, path + "uncertainty/sensor/"+Sensor+"_"
                                                         +str(model_chosen[0])+"_"+str(model_chosen[1])+"_"+str(model_chosen[2])
                                                     +"_"+str(model_chosen[3])+"_"+str(model_chosen[4])+".csv")

        print("Interpolation finished !")

../data/weather_2022/Quebec/data_sorted_2022.txt
[1, 2, 1, 1, 3, 2]
Number of data for theodolites: [4945 4681 4737]
Bad measures: 558
3836 3619 3690
Number of sub-trajectories : 47
Étape 0
Number of input:  86
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  100.7
Étape 1
Number of input:  17
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  17.7
Étape 2
Number of input:  20
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  25.8
Étape 3
Number of input:  54
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  62.55
Étape 4
Number of input:  112
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  127.47
Étape 5
Number of input:  50
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  57.05
Étape 6
Number of input:  141
Interpolation MC done !
Interpolation MC done !
Interpolation MC done !
Time :  187.96
Étape 7
Num

In [ ]:
theodo_g = importlib.reload(theodo_g)
%matplotlib notebook
plt.figure()
for i in M_2_before_inter:
    theodo_g.plot_ellipse(plt.gca(), i[1][0:2], i[2][0:2,0:2], n_std=1, color="red")
    plt.scatter(i[1][0], i[1][1], s=2, color='r')
for i in MC_2_inter[0:-1]:
    theodo_g.plot_ellipse(plt.gca(), i[1][0:2], i[2][0:2,0:2], n_std=1, color="green")
    plt.scatter(i[1][0], i[1][1], s=2, color='green')
plt.show()

In [ ]:
theodo_g = importlib.reload(theodo_g)
%matplotlib notebook
plt.figure()
for i in M_1_before_inter:
    theodo_g.plot_ellipse(plt.gca(), i[1][1:3], i[2][1:3,1:3], n_std=1, color="red")
    plt.scatter(i[1][1], i[1][2], s=2, color='r')
for i in MC_1_inter[0:-1]:
    theodo_g.plot_ellipse(plt.gca(), i[1][1:3], i[2][1:3,1:3], n_std=1, color="green")
    plt.scatter(i[1][1], i[1][2], s=2, color='green')
plt.show()

In [ ]:
timestamp = []
Coordinates = []
Covariance = []
for i in MC_1_sorted:
    timestamp.append(i[0])
    Coordinates.append(i[1][0:3])
    cov = i[2][0:3,0:3]
    w, v = np.linalg.eig(cov)
    Covariance.append(w)
timestamp = np.array(timestamp)
Coordinates = np.array(Coordinates)
Covariance = np.array(Covariance)

tp1c = (Tf_1@tp1).T
tp2c = (Tf_2@tp2).T
tp3c = (Tf_3@tp3).T

tp1c = tp1.T
tp2c = tp2.T
tp3c = tp3.T

In [ ]:
%matplotlib notebook
tp_data = tp1c
t_data = t1
std_n = 10
limit = 300
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(6, 6))
ax1.scatter(t_data[:limit], tp_data[:limit,0], color='black', s=2)
ax1.scatter(timestamp, Coordinates[:,0], color='green', s=2)
ax1.fill_between(timestamp, Coordinates[:,0]-std_n*Covariance[:,0], Coordinates[:,0]+std_n*Covariance[:,0], color="green", alpha=0.3)
ax2.scatter(t_data[:limit], tp_data[:limit,1], color='black', s=2)
ax2.scatter(timestamp, Coordinates[:,1], color='red', s=2)
ax2.fill_between(timestamp, Coordinates[:,1]-std_n*Covariance[:,1], Coordinates[:,1]+std_n*Covariance[:,1], color="red", alpha=0.3)
ax3.scatter(t_data[:limit], tp_data[:limit,2], color='black', s=2)
ax3.scatter(timestamp, Coordinates[:,2], color='blue', s=2)
ax3.fill_between(timestamp, Coordinates[:,2]-std_n*Covariance[:,2], Coordinates[:,2]+std_n*Covariance[:,2], color="blue", alpha=0.3)
plt.show()

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
theodo_g = importlib.reload(theodo_g)

# Setup the plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plt.xlabel('x')
plt.ylabel('y')

for info in MC_1_sorted:
    mu1= info[1]
    cov1 = info[2][0:3,0:3]
    s1 = np.random.multivariate_normal(mu1, cov1, (200))
    nstd = 2    # 95% confidence interval
    mu1_ = np.mean(s1, axis=0)
    cov1_ = np.cov(s1.T)
    X1,Y1,Z1 = theodo_g.get_cov_ellipsoid_bis(cov1_, mu1_, nstd)
    ax.plot_wireframe(X1,Y1,Z1, color='r', alpha=0.1)
for info in MC_2_sorted:
    mu1= info[1]
    cov1 = info[2][0:3,0:3]
    s1 = np.random.multivariate_normal(mu1, cov1, (200))
    nstd = 2    # 95% confidence interval
    mu1_ = np.mean(s1, axis=0)
    cov1_ = np.cov(s1.T)
    X1,Y1,Z1 = theodo_g.get_cov_ellipsoid_bis(cov1_, mu1_, nstd)
    ax.plot_wireframe(X1,Y1,Z1, color='green', alpha=0.1)
plt.show()

In [ ]:
theodo_u = importlib.reload(theodo_u)

path_vtk = "/home/maxime/data/vtk_traj/20220224/"
sigma_plot = 10
theodo_u.save_to_VTK_uncertainty(sigma_plot, MC_1_sorted, path_vtk + "Prisme_1.vtk")
theodo_u.save_to_VTK_uncertainty(sigma_plot, MC_2_sorted, path_vtk + "Prisme_2.vtk")
theodo_u.save_to_VTK_uncertainty(sigma_plot, MC_3_sorted, path_vtk + "Prisme_3.vtk")
theodo_u.save_to_VTK_uncertainty(sigma_plot, Pose_sensor_MC, path_vtk + "Sensor.vtk")
print("Done !")

In [ ]:
Mu_1 = MC_1_sorted[2][1]
Mu_2 = MC_1_sorted[1][1]
C1 = MC_1_sorted[2][2]
C2 = MC_1_sorted[1][2]
print(Mu_1)
print(Mu_2)
print(C1)
print(C2)

In [ ]:
theodo_u = importlib.reload(theodo_u)
# Bhattacharyya distance, measures the similarity of two probability distributions, not a metric
print(theodo_u.Bhattacharyya_distance(Mu_1, Mu_2, C1, C2))
# Hellinger distance, quantify the similarity between two probability distributions, bounded metric ([0,1])
print(theodo_u.Hellinger_distance_square(Mu_1, Mu_2, C1, C2))